In [ ]:
#| default_exp data.utils

# 2) Download utilities
> Utils for downloading and preprocessing data

In [ ]:
#| hide

import os

In [ ]:
#| hide

os.chdir('../')

In [ ]:
#| export
#| hide

import os
from time import time
import shutil
from typing import List, Tuple

In [ ]:
#| export

def timer_func(func):
    """Show the execution time of the function object passed.

    Parameters
    ----------
    func : _type_
        _description_
    """

    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f"Function {func.__name__!r} executed in {(t2-t1):.4f}s")
        return result

    return wrap_func

Recursively finding all files in folder

In [ ]:
#| export

def find_filepaths(root_folder: str) -> (List, int):
    """Recursively finds all files.

    Parameters
    ----------
    root_folder : str
        folder path

    Returns
    -------
    Tuple
        Returns a Tuple (Sorted filepaths, length of filepaths)
    """
    filepaths = []
    for dirname, _, filenames in os.walk(root_folder):
        for filename in filenames:
            filepaths.append(os.path.join(dirname, filename))

    return sorted(filepaths), len(filepaths)

In [ ]:
#| export

def move_to_processed():
    """
    Combines files with same subclass and moves them to the subclass under data/2_processed.

    Uses `get_new_name` to create new names of files and then rename them and copy to data/2_processed.
    """
    dir1 = "data/1_extracted/dataset1/"
    dir2 = "data/1_extracted/dataset2/"
    for d1, d2 in zip(sorted(os.listdir(dir1)), sorted(os.listdir(dir2))):
        path_dict = get_new_name([os.path.join(dir1, d1), os.path.join(dir2, d2)])

        for old_path, new_path in path_dict.items():
            shutil.copy(old_path, new_path)


def get_new_name(dir_list: list) -> dict:
    """Return dict with old name and new name of files in multiple directories.

    {'data/1_extracted/dataset1/Basalt/14.jpg': 'data/2_processed/Basalt/dataset1_01_Basalt_14.jpg'}

    Parameters
    ----------
    dir_list : list
        list of dir paths

    Returns
    -------
    dict
        {old_name: new_name}
    """
    file_list = []
    for dir in dir_list:
        paths, _ = find_filepaths(dir)
        file_list.extend(paths)

    count = 1
    file_dict = {}
    for file_path in file_list:
        dataset = file_path.split("/")[-3]
        class_name = file_path.split("/")[-2]
        basename = os.path.basename(file_path)
        file_name = os.path.splitext(basename)[0]
        extension = os.path.splitext(basename)[1]
        new_file_name = os.path.join(
            "data",
            "2_processed",
            class_name,
            f"{dataset}_{class_name}_{str(count).zfill(3)}_{file_name}{extension}",
        )
        file_dict[file_path] = new_file_name
        count += 1

    return file_dict